# Cohort Case Study 2

In [1]:
import sys
sys.path.append('../../../notebooks')

import random
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

from raimitigations.utils import split_data, fetch_cohort_results
import raimitigations.dataprocessing as dp
from raimitigations.cohort import CohortDefinition, CohortManager
from download import download_datasets

SEED = 46

np.random.seed(SEED)
random.seed(SEED)

data_dir = "../../../datasets/"
download_datasets(data_dir)

df = pd.read_csv(data_dir + "AdultCensusIncome.csv", skipinitialspace=True)
label_col = "income"
# convert to 0 and 1 encoding
df[label_col] = df[label_col].apply(lambda x: 0 if x == "<=50K" else 1)

In [2]:
df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,0
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,1
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,0
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,0


In [3]:
X_train, X_test, y_train, y_test = split_data(df, label=label_col, test_size=0.25)

cht_manager = CohortManager(cohort_col=["race"])
cht_manager.fit(X_test, y_test)
subsets = cht_manager.get_subsets(X_test, y_test)
queries = cht_manager.get_queries()
for key in subsets.keys():
    print(f"{queries[key]}:\n {subsets[key]['y'].value_counts()}\n")

(`race` == "Amer-Indian-Eskimo"):
 0    71
1    10
Name: income, dtype: int64

(`race` == "Asian-Pac-Islander"):
 0    195
1     78
Name: income, dtype: int64

(`race` == "Black"):
 0    706
1     98
Name: income, dtype: int64

(`race` == "Other"):
 0    60
1    10
Name: income, dtype: int64

(`race` == "White"):
 0    5149
1    1764
Name: income, dtype: int64



In [4]:
model = LGBMClassifier(random_state=SEED)
pipe = Pipeline([
            ("imputer", dp.BasicImputer(verbose=False)),
            ("scaler", dp.DataStandardScaler(verbose=False)),
            ("encoder", dp.EncoderOHE(verbose=False)),
            ("estimator", model),
        ])
pipe.fit(X_train, y_train)
pred = pipe.predict_proba(X_test)
    
experiments = {}
experiments["Baseline (race)"] = fetch_cohort_results(X_test, y_test, pred, cohort_col=["race"])
experiments["Baseline (race)"]

,cohort,cht_query,cht_size,roc,pr,recall,f1,acc
0,all,all,8141,0.926859,0.774437,0.846028,0.792408,0.825451
1,cohort_0,"(`race` == ""Amer-Indian-Eskimo"")",81,0.930986,0.716803,0.872535,0.754545,0.851852
2,cohort_1,"(`race` == ""Asian-Pac-Islander"")",273,0.893097,0.781805,0.825641,0.793956,0.816850
3,cohort_2,"(`race` == ""Black"")",804,0.958866,0.752209,0.915852,0.797888,0.883085
4,cohort_3,"(`race` == ""Other"")",70,0.928333,0.771991,0.891667,0.811066,0.885714
5,cohort_4,"(`race` == ""White"")",6913,0.924036,0.774911,0.840937,0.789829,0.817735


In [5]:
experiments["Baseline (sex)"] = fetch_cohort_results(X_test, y_test, pred, cohort_col=["sex"])
experiments["Baseline (sex)"]

,cohort,cht_query,cht_size,roc,pr,recall,f1,acc
0,all,all,8141,0.926859,0.774437,0.846028,0.792408,0.825451
1,cohort_0,"(`sex` == ""Female"")",2700,0.946919,0.717245,0.883079,0.761776,0.875926
2,cohort_1,"(`sex` == ""Male"")",5441,0.906967,0.773631,0.815405,0.778935,0.793053


In [6]:
experiments["Baseline (relationship)"] = fetch_cohort_results(X_test, y_test, pred, cohort_col=["relationship"])
experiments["Baseline (relationship)"]

/home/mmendonca/ResponsibleAI/code/git/responsible-ai-toolbox-mitigations-private/raimitigations/utils/model_utils.py:140: RuntimeWarning: invalid value encountered in true_divide
  fscore = (2 * precision * recall) / (precision + recall)


,cohort,cht_query,cht_size,roc,pr,recall,f1,acc
0,all,all,8141,0.926859,0.774437,0.846028,0.792408,0.825451
1,cohort_0,"(`relationship` == ""Husband"")",3304,0.856983,0.770280,0.773238,0.770890,0.772397
2,cohort_1,"(`relationship` == ""Not-in-family"")",2110,0.915764,0.677099,0.839903,0.707384,0.824171
3,cohort_2,"(`relationship` == ""Other-relative"")",232,0.873326,0.583185,0.812500,0.607312,0.870690
4,cohort_3,"(`relationship` == ""Own-child"")",1270,0.963400,0.589217,0.899378,0.629583,0.922835
5,cohort_4,"(`relationship` == ""Unmarried"")",864,0.939461,0.679695,0.865079,0.726035,0.885417
6,cohort_5,"(`relationship` == ""Wife"")",361,0.827201,0.760314,0.759252,0.753432,0.753463
